## **SCOTIA Analysis of CCI**

*   The SCOTIA documentation and package could be found  here:
https://github.com/Caochris/SCOTIA
*   The paper could be found here:
https://www.nature.com/articles/s41588-024-01890-9


### 1. **Import Package**

In [ ]:
import scotia
import numpy as np
import pandas as pd
import seaborn as sns
import scanpy as sc
import anndata as ad
import matplotlib.pyplot as plt
from matplotlib import cm
from scipy.spatial import distance_matrix
from scipy.stats import ranksums
from matplotlib.collections import LineCollection
from statsmodels.stats.multitest import multipletests
import itertools
import warnings
warnings.filterwarnings("ignore")
import sys

### 2. **Run the Analysis**



*   The below shows an example analysis on analyzing spot level data, where I have prepared the data by the following columns: `["x", "y", "celltypeA",..., "celltypeX", "cell_type(dominant cell type)", "GeneA",..., "GeneX"]`.
*   According to their paper, they use the LR pairs from the [FANTOM5 database](https://doi.org/10.1038/ncomms8866)
*   According to their paper, they do the permutation for 50 times, but we increase to 500 times.






In [ ]:
for num in ["slice1"]:
  path_exprsn = f'/rsrch5/home/biostatistics/lku/ILIBD/data/{num}/exprsn_df.csv'

  data = pd.read_csv(path_exprsn, index_col=0)
  data = data.dropna(subset=['cell_type'])
  gene_expression= data.drop(columns=["x","y","cell","Excitatory_neurons","Inhibitory_neuron", "Astrocyte", "Oligodendrocyte","Oligodendrocyte_precursor_cell",
                    "Microglia","Pericytes", "Endothelial_cells","cell_type" ])
  meta_data = data[["cell","x","y","cell_type"]]
  meta_data.loc[:,'cell_id'] = meta_data['cell']
  meta_data.set_index('cell', inplace=True)
  meta_data['fov'] = 0
  meta_data.loc[:,'x_positions'] = meta_data['x']
  meta_data.loc[:,'y_positions'] = meta_data['y']
  meta_data.loc[:, 'x_pos'] = meta_data['x']
  meta_data.loc[:, 'y_pos'] = meta_data['y']
  meta_data.loc[:, 'sample'] = "test"
  meta_data.loc[:,'annotation'] = meta_data['cell_type']
  meta_df = meta_data[["cell_id", "fov"   , "annotation" , "x_positions" , "y_positions",'sample','x_pos','y_pos']]
  cell_types = []
  cell_indices = []
  meta_df_fov = meta_df
  meta_df_fov['index'] = range(meta_df_fov.shape[0])
  cell_type_l = list(set(meta_df_fov['annotation']))


  # Generate the Null Distribution

  for ct in cell_type_l:
    meta_df_sel = meta_df_fov[meta_df_fov['annotation']==ct]
    idx_l = np.array(meta_df_sel['index'])
    cell_types.append(ct)
    cell_indices.append(list(idx_l))
  # Create the resulting DataFrame
  cluster_cell_df = pd.DataFrame({'cell_type': cell_types, 'cell_idx': cell_indices})
  coord = np.array(meta_df[['x_positions','y_positions']])
  S_all_arr = distance_matrix(coord,coord)
  dis_mtx_mod = scotia.sel_pot_inter_cluster_pairs(S_all_arr,cluster_cell_df)
  exp_df = gene_expression
  exp_df['cell_id'] = exp_df.index
  exp_df['fov'] = 0
  first_columns = ['cell_id', 'fov']

  # Reorder the columns: first_columns + all other columns not in first_columns
  exp_df = exp_df[first_columns + [col for col in exp_df.columns if col not in first_columns]]
  exp_df = exp_df.reset_index(drop=True)
  meta_df1 = meta_df[['cell_id','fov','annotation','x_positions','y_positions']]
  meta_df1 = meta_df1.reset_index(drop=True)
  #known LR pairs
  known_lr_pairs = pd.read_csv('/rsrch5/home/biostatistics/lku/FANTOM5_db.csv', index_col=0)
  known_lr_pairs = known_lr_pairs.reset_index(drop=True)
  valid_genes = set(exp_df.columns)
  known_lr_pairs1 = known_lr_pairs[known_lr_pairs['l_gene'].isin(valid_genes)]
  known_lr_pairs = known_lr_pairs1[known_lr_pairs1['r_gene'].isin(valid_genes)]
  ## infer the null ot
  inter_likely_df = scotia.source_target_ot(dis_mtx_mod, exp_df, meta_df1, known_lr_pairs)
  inter_likely_df.columns = ['source_cell_idx','receptor_cell_idx','likelihood','ligand_recptor','source_cell_type','target_cell_type']
  inter_likely_df['cell_pairs'] = inter_likely_df['source_cell_type']+"-"+ inter_likely_df['target_cell_type']
  ## infer the null avg likelihood
  group = "test"
  inter_likely_df['cell_pairs'] = inter_likely_df['source_cell_type']+"_"+ inter_likely_df['target_cell_type']
  null_final_summary = scotia.post_ot(inter_likely_df,label=group)
  null_final_summary.to_csv(f'/rsrch5/home/biostatistics/lku/ILIBD/data/{num}/null_ot_SCOTIA_result.csv', index=False)

  #permutation test
  #gene expression normalization factor
  gene_expression= data.drop(columns=["x", "y", "cell_type", "cell"])
  exp_df_all = gene_expression
  exp_df_norm = exp_df_all.iloc[:,0:]
  exp_df_norm = exp_df_norm[exp_df_norm>0]
  df_quantile = exp_df_norm.quantile(q=0.99,axis = 0)
  fov = 0
  cluster_df = cluster_cell_df
  cluster_df.columns = ['cell_type','cell_idx']

  #coordinates
  meta_df_fov = meta_df1[meta_df1['fov']==fov]
  meta_df_fov.index = range(meta_df_fov.shape[0])
  cell_id_all = np.array(range(meta_df_fov.shape[0]))
  coord = np.array(meta_df_fov[['x_positions','y_positions']])
  S_all_arr = distance_matrix(coord,coord)

  #expression
  exp_df_fov = exp_df[exp_df['fov']==fov].iloc[:,2:]
  exp_df_fov = exp_df_fov/df_quantile
  exp_df_fov[exp_df_fov>1] = 1
  exp_df_fov.index = cell_id_all

  # Generate the Permutation Coordinates and Cell Types
  it_n = 500
  #get permutated positions and expression
  random_pos, shuffled_exp = scotia.permutation_test(coord,int(it_n))
  #select potentially communicating cell cluster pairs (spatially adjacent)
  S_all_arr_new = scotia.sel_pot_inter_cluster_pairs(S_all_arr,cluster_df)

  final_summary = pd.DataFrame([])
  for i_n in range(int(it_n)):
    exp_df_permu = exp_df_fov.loc[list(shuffled_exp.iloc[:,i_n])]
    coord_permu = np.array(random_pos.iloc[:,i_n*2:i_n*2+2])
    S_all_arr_permu = distance_matrix(coord_permu,coord_permu)
    mask = np.where(S_all_arr_new==np.inf)
    S_all_arr_permu[mask] = np.inf
    # optimal transport
    ga_df_permu = scotia.source_target_ot(S_all_arr_permu, exp_df_permu, meta_df_fov, known_lr_pairs)
    if ga_df_permu.shape[0]>0:
      ga_df_permu.columns = ['source_cell_idx','receptor_cell_idx','likelihood','ligand_recptor','source_cell_type','target_cell_type']

      #post-processing ot results by calculating averaged likelihoods
      ga_df_permu['cell_pairs'] = ga_df_permu['source_cell_type']+"_"+ga_df_permu['target_cell_type']
      final_summary_tmp = scotia.post_ot(ga_df_permu, label=group, it_n_label = i_n)
      final_summary = pd.concat([final_summary,final_summary_tmp])
      final_summary.to_csv(f'/rsrch5/home/biostatistics/lku/ILIBD/data/{num}/permu_ot_SCOTIA_result.csv', index=False)

  ###########summary for permutation test ####################
  fov = 0
  it_n = 500
  group = 'test'

  df_ot = pd.DataFrame([])
  df_permu = pd.DataFrame([])

  compare_dic ={}
  #import ot result
  df_ot_tmp = null_final_summary
  if df_ot_tmp.shape[0]>0:
    df_ot = pd.concat([df_ot,df_ot_tmp])
  #import permutation result
  df_permu_tmp = final_summary
  if df_permu_tmp.shape[0]>0:
    df_permu = pd.concat([df_permu,df_permu_tmp])
  df_ot['lr_pairs'] = [x.split('|')[0] for x in df_ot['label']]
  df_ot['cell_pairs'] = [x.split('|')[2] for x in df_ot['label']]

  df_permu['it_n'] = [x.split('|')[-1] for x in df_permu['label']]
  df_permu['lr_pairs'] = [x.split('|')[0] for x in df_permu['label']]
  df_permu['label_new'] = df_permu['lr_pairs']+"|"+df_permu['it_n'].map(str)
  df_permu['cell_pairs'] = [x.split('|')[2] for x in df_permu['label']]
  # Assuming meta_df1 contains the annotation column with unique cell types
  unique_cell_types = meta_df1['annotation'].unique()

  # Initialize the larger DataFrame to store all results
  result_df = pd.DataFrame()
  # Iterate over all combinations of c_t_1 and c_t_2
  for c_t_1, c_t_2 in itertools.product(unique_cell_types, repeat=2):
    # Initialize variables specific to the combination
    c_t_p = c_t_1 + "_" + c_t_2
    df_ot_tmp = df_ot[df_ot['cell_pairs'] == c_t_p]
    df_permu_tmp = df_permu[df_permu['cell_pairs'] == c_t_p]
    # Dictionary to store comparisons
    compare_dic = {}
    if df_permu_tmp.shape[0] > 0:
      df_ot_groupby = df_ot_tmp.groupby(['label'])['ave_likelihood'].sum().to_frame().reset_index()
      df_ot_groupby.index = [x.split('|')[0] for x in df_ot_groupby['label']]
      df_permu_groupby = df_permu_tmp.groupby(['label_new'])['ave_likelihood'].sum()
      for i in df_ot_groupby.index:
        compare_dic[i + "|" + group + "|" + c_t_p] = 0
        df_permu_groupby_sel = df_permu_groupby[df_permu_groupby.index.str.contains(i + r"\|")]
        for j in df_permu_groupby_sel:
          if j > df_ot_groupby.loc[i, 'ave_likelihood']:
            compare_dic[i + "|" + group + "|" + c_t_p] += 1 / it_n
      p_l = [round(x, 3) for x in compare_dic.values()]
      # Adjust p-values using FDR correction
      if len(p_l) > 0:
        pvalue_adj = multipletests(p_l, alpha=0.05, method='fdr_bh')[1]
        pvalue_adj = [round(x, 3) for x in pvalue_adj]
        temp_df = pd.DataFrame({
            'comparison': compare_dic.keys(),
            'p_value': p_l,
            'p_value_adj': pvalue_adj,
            'c_t_p': c_t_p
            })
         #Append to the larger result DataFrame
        result_df = pd.concat([result_df, temp_df], ignore_index=True)
      else:
        print(f"No significant comparisons for {c_t_p}. Skipping.")
  # Final DataFrame with all results
  result_df.reset_index(drop=True, inplace=True)
  # Split the "comparison" column by "|" and extract the first element as "interaction_name"
  result_df['interaction_name'] = result_df['comparison'].str.split('|').str[0]
  # Split the "c_t_p" column by "_" to extract "ligand" and "receptor"
  def split_c_t_p(value):
    parts = value.split('_', 1)  # Split only at the first "_"
    if len(parts) == 2:
      return parts[0], parts[1]
    else:
      return parts[0], None
  result_df[['ligand', 'receptor']] = result_df['c_t_p'].apply(split_c_t_p).apply(pd.Series)
  result_df['pvalue'] = result_df['p_value_adj']
  merged_df = pd.merge(result_df, null_final_summary, left_on='comparison', right_on='label', how='left')
  final_summary = merged_df[['ligand','receptor','interaction_name','ave_likelihood','pvalue']]
  path_write_LR = f'/rsrch5/home/biostatistics/lku/ILIBD/data/{num}/scotia_result.csv'
  final_summary.to_csv(path_write_LR , index=False)